In [9]:
:dep graph-ds = { path = "." }
:dep h3o = "0.3.0"
:dep plotters = "0.3.4"

In [12]:
use graph_ds::{Graph, Node, Edge};
use graph_ds::hexagon_graph::{hexagon_graph_from_file, cell::Cell, h3_network_from_gtfs, h3_network_from_osm, osm::{process_osm_pbf, OSMLayer}};
use h3o::Resolution;
use std::time::Instant;

In [13]:
let osm_graph = h3_network_from_osm("../resources/denmark-with-ways.osm.pbf", OSMLayer::Walking).unwrap();
println!("osm graph created with {} nodes", osm_graph.nr_nodes())

converted OSM file into 14263558 edges


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

In [ ]:
let gtfs_graph = h3_network_from_gtfs("../resources/rejseplanen.zip").unwrap();
println!("gtfs graph created with {} nodes", gtfs_graph.nr_nodes())

In [ ]:
let mut graph : Graph<Cell> = hexagon_graph_from_file("de_inferno_hexagons.mpk")?;
println!("Graph has {} nodes", graph.nr_nodes());

In [ ]:
let start = Cell {
    a: 60,
    b: -33,
    radius: 24,
    layer: 3,
};
let end = Cell {
    a: 5,
    b: 61,
    radius: 24,
    layer: 3,
};

In [ ]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [ ]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (path, _) = graph.bfs(start, Some(end))?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", path.unwrap().len());

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (_, distance) = graph.astar(start, end, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", distance);